In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib widget
import pickle as pkl
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient


In [ ]:
client = EfdClient('summit_efd')

In [ ]:
# Times to start looking at error codes
start = Time("2022-06-29 00:00:00Z", scale='utc')
end = Time("2022-07-01 10:00:00Z", scale='utc')

In [ ]:
# Observing runs
day_start = []
day_end = []
day_start.append(Time("2022-06-29 00:00:00Z", scale='utc'))
day_end.append(Time("2022-06-29 10:00:00Z", scale='utc'))
day_start.append(Time("2022-06-30 00:00:00Z", scale='utc'))
day_end.append(Time("2022-06-30 08:00:00Z", scale='utc'))
day_start.append(Time("2022-07-01 00:00:00Z", scale='utc'))
day_end.append(Time("2022-07-01 10:00:00Z", scale='utc'))

In [ ]:
filterInPosition = await client.select_time_series('lsst.sal.ATSpectrograph.logevent_filterInPosition', \
                                                'inPosition',  start, end)
filterInPosition = filterInPosition[filterInPosition['inPosition']==True]

In [ ]:
len(filterInPosition)

In [ ]:
filterInPosition.tail(20)

In [ ]:
changeFilter = await client.select_time_series('lsst.sal.ATSpectrograph.command_changeFilter', \
                                                'filter',  start, end)
#filterInPosition = filterInPosition[filterInPosition['inPosition']==True]

In [ ]:
len(changeFilter)

In [ ]:
domePosition = await client.select_time_series('lsst.sal.ATDome.position', \
                                                'azimuthPosition',  start, end)

In [ ]:
len(domePosition)

In [ ]:
domeInPosition = await client.select_time_series('lsst.sal.ATDome.logevent_azimuthInPosition', \
                                                'inPosition',  start, end)
domeInPosition = domeInPosition[domeInPosition['inPosition']==True]

In [ ]:
len(domeInPosition)

In [ ]:
domeInPosition.head(10)

In [ ]:
mountFault = await client.select_time_series('lsst.sal.ATMCS.logevent_summaryState', \
                                                'summaryState',  start, end)
mountFault = mountFault[mountFault['summaryState']==3]

In [ ]:
len(mountFault)

In [ ]:
mountFault

In [ ]:
fig = plt.figure(figsize = (8,11))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"Mount Errors - JuneB Run", fontsize = 16)
for j in range(3):
    plt.subplot(3,1,j+1)
    plt.title("Day %d"%(j+1))
    ax = domePosition['azimuthPosition'].plot(color='red')
    for i in range(len(mountFault)):
        ax.axvline(mountFault.index[i], color="green", linestyle="--", label="Mount Fault")
    #for k in range(len(domeInPosition)):
    #    ax.axvline(domeInPosition.index[k], color="blue", linestyle="--", label="Dome in Position")
    ax.set_ylabel("Dome Azimuth (degrees)")
    ax.set_xlim(day_start[j].to_datetime(), day_end[j].to_datetime())
plt.savefig("/home/craiglagegit/DATA/Mount_Faults_Dome_Motion_June2022.pdf")

In [ ]:
fig = plt.figure(figsize = (8,11))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"Mount Errors - JuneB Run", fontsize = 16)
for j in range(3):
    plt.subplot(3,1,j+1)
    plt.title("Day %d"%(j+1))
    ax = domePosition['azimuthPosition'].plot(color='red')
    for i in range(len(mountFault)):
        ax.axvline(mountFault.index[i], color="green", linestyle="--", label="Mount Fault")
    for k in range(len(filterInPosition)):
        ax.axvline(filterInPosition.index[k], color="blue", linestyle="--", label="Dome in Position")
    ax.set_ylabel("Dome Azimuth (degrees)")
    
    plotStart = Time("2022-06-29 04:30:00Z", scale='utc')
    plotEnd = Time("2022-06-29 04:40:00Z", scale='utc')
    ax.set_xlim(plotStart.to_datetime(), plotEnd.to_datetime())
#plt.savefig("/home/craiglagegit/DATA/Mount_Faults_Dome_Motion_June2022.pdf")

In [ ]:
day_start